# **Vector databases for embedding systems**

<img src = './images/limits-of-current-approach.png' width=50% height=50%>

So far, we've created embeddings using the OpenAI API and stored them in-memory. 

* Loading all the embeddings into memory (1536 floats ~ 13kB/embedding), which becomes impractical to load for 100,000s or millions of embeddings. 
* Recalculated these embeddings with every query rather than storing them for later use. 
* We computed cosine distances for every embedded document and sorted the results, which are both slow processes that scale linearly. 

To enable embeddings applications with larger datasets in production, we'll need a better solution: __vector databases__!

## **Vector databases**

Here's a typical embeddings application: 

<img src = './images/embedding-app-arch.png' width=50% height=50%>

* Embedded documents are _stored_ and _queried_ from the __vector database__

- The documents to query are embedded and stored in the vector database. 
- A query is sent from the application interface, embedded, and used to query the embeddings in the database. This query can be a semantic search query or data to base recommendations on. 
- Finally, these results are returned by to the user via the application interface. 

Because the embedded documents are stored in the vector database, they don't have to created with each query or stored in-memory. Additionally, due to the architecture of the database, the similarity calculation is computed much more efficiently.

### **NoSQL databases vs SQL databases**

The majority of vector databases are what's called NoSQL databases, which contrasts conventional databases.

<div style="display: flex;">
    <div style="flex: 50%; padding: 5px; border-right: 2px solid DodgerBlue;">
        **NoSQL Database**  <br>
        <div style="text-align: center;">
            <img src='./images/nosql-db.png' width=75% height=80%>
        </div>
        NoSQL databases don't use tables
        <ul>
            <li>More flexible structure that allows for <i>faster querying</i></li>
            <li>Three examples are shown above: including key:value, document, and graph databases</li>
        </ul>
    </div>
    <div style="flex: 50%; padding: 5px;">
        **SQL/Relational Database**  <br>
        <div style="text-align: center;">
            <img src='./images/sql-db.png' width=75% height=80%>
        </div>
        <ul>
            <li>Structured data into tables, rows. and columns</li>
        </ul>
    </div>
</div>

### **The vector database landscape**

<img src='./images/vector-db-options.png' width=60% height=60%>

When deciding which database solution to go with, there are several factors to consider.

### **Which solution is best?**

<div style="display: flex;">
    <div style="flex: 50%; padding: 5px; border-right: 2px solid DodgerBlue;">
        <ul>
           <li><b>Database management</b></li>
           <ul>
               <li>Managed &#x2192; more expensive but lowers workload.</li>
               <li>Self-managed &#x2192; cheaper but requires time and expertise
            </ul>
        </ul>
        <ul>
            <li><b>Open source or commercial?</b></li>
            <ul>
               <li>Open source &#x2192; flexible and cost-effective if budgets are tight</li>
               <li>Commercial &#x2192; offers better support, more advanced features, and compliance</li>
        </ul>
    </div>
    <div style="flex: 50%; padding: 5px;">
        <ul>
            <li><b>Data models</b>: does the type of data lend itself to a particular database type?</li>
            <li><b> Specific features</b>: does your use case depend on specific functionality, like embedding and storung both text and images for a multi-modal application?</li>
        </ul>
        In this course, we'll be using Chroma, as it's open-source and quick to set up.
        <div style="text-align: center;">
            <img src='./images/chroma.png' width=50% height=50%>
        </div>
    </div>
</div>



## **Creating vector databases with ChromaDB**